<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fa017dfd6ace1d747d6e79013be0462da999a8852be67febf24c01e9d4d9427e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 15:20:07
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -21.72 K (-0.15%)
Current PnL: -22.35 L (-14.65%)
CY Booked + Current PnL: -8.46 L (-5.54%)
-------------------
Total profit:  1.76 L
Total loss:  -24.11 L
-------------------
Total Booked + Current PnL: 18.52 L (14.74%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.68%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.76 L (61.18%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.53%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,0.14,-14.00,16.38,0.09,14197.0,-14107.0,86670.0,101.07,51.0,M-SC,3.55,253.0,-0.99,0.60,13.86,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-2.20,-9.22,10.18,0.02,16132.0,-16093.0,158466.0,141.63,66.0,M-SC,9.14,234.0,-1.00,1.10,37.84,OX40N,NTT,PAINTS
83,VOLTAS,1530.0,0.87,9.45,9.36,19.69,19643.0,18123.0,209865.0,-1.22,55.0,X-MC,3.37,78.0,0.92,1.46,16.89,XY25,NTT,AC
43,ITC,452.0,0.42,-1.29,11.88,10.44,23470.0,-2582.0,197556.0,-41.15,45.0,X-LC,1.01,5.0,-0.11,1.38,4.20,X40,NTT,FMCG
52,MEDANTA,1486.0,-1.18,4.24,19.03,24.08,25182.0,5380.0,132330.0,-4.20,48.0,X-SC,5.56,89.0,0.21,0.92,24.69,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,4.24,3.80,27.57,32.42,72661.0,9654.0,263550.0,-14.60,52.0,X-SC,7.53,85.0,0.13,1.84,19.48,X40,ATH,FMCG
58,RAJOOENG,143.10,3.41,-24.89,85.87,39.61,66111.0,-25510.0,76990.0,-49.12,39.0,H-SC,20.48,135.0,-0.39,0.54,8.51,AR,ATH,MISC
67,SIS,528.00,2.44,-20.98,53.47,21.26,46911.0,-23299.0,87733.0,2074.09,61.0,H-SC,3.79,166.0,-0.50,0.61,18.54,OX40N,NTT,MISC
81,VALIANTORG,838.00,2.43,-46.11,223.18,74.15,196760.0,-75443.0,88162.0,-258.45,31.0,H-SC,17.04,149.0,-0.38,0.61,13.01,XR,NTT,CHEMICALS
50,LTIM,7201.88,2.19,10.04,19.65,31.66,53223.0,24704.0,270855.0,5.72,64.0,H-LC,13.21,43.0,0.46,1.89,50.28,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-11.56,-14.88,113.73,81.92,111129.0,-17085.0,97713.0,-50.12,23.0,M-SC,3.32,236.0,-0.15,0.68,15.68,XR,NTT,DURABLES
44,JCHAC,2282.0,-3.32,-27.30,37.54,-0.01,34878.0,-34895.0,92910.0,18921.74,17.0,M-SC,1.44,233.0,-1.00,0.65,5.21,OX40N,NTT,AC
39,INDIGOPNTS,1408.0,-2.20,-9.22,10.18,0.02,16132.0,-16093.0,158466.0,141.63,66.0,M-SC,9.14,234.0,-1.00,1.10,37.84,OX40N,NTT,PAINTS
28,GREENPANEL,537.0,-2.11,-32.62,112.42,43.14,138688.0,-59712.0,123366.0,176.49,32.0,M-SC,5.86,240.0,-0.43,0.86,16.04,XY24,NTT,MISC
2,ABBOTINDIA,35195.0,-2.05,-1.87,19.67,17.44,28925.0,-2797.0,147050.0,-15.54,45.0,X-MC,4.28,63.0,-0.10,1.03,16.93,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.5,0.04,106.65,106.72,131584.0,43.0,123379.0,-50.44,38.0,H-SC,3.11,139.0,0.0,0.86,23.42,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.47,-4.65,118.04,107.91,166180.0,-6861.0,140783.0,-22.58,36.0,M-SC,10.57,216.0,-0.04,0.98,0.67,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.20,-9.22,10.18,0.02,16132.0,-16093.0,158466.0,141.63,66.0,M-SC,9.14,234.0,-1.00,1.10,37.84,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.14,-14.00,16.38,0.09,14197.0,-14107.0,86670.0,101.07,51.0,M-SC,3.55,253.0,-0.99,0.60,13.86,OX40N,NTT,DURABLES
47,KANSAINER,340.00,1.21,-20.96,43.56,13.47,92847.0,-56520.0,213147.0,-67.53,39.0,H-SC,1.47,158.0,-0.61,1.49,8.38,XY24,NTT,PAINTS
67,SIS,528.00,2.44,-20.98,53.47,21.26,46911.0,-23299.0,87733.0,2074.09,61.0,H-SC,3.79,166.0,-0.50,0.61,18.54,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.18,2.43,68.18,72.27,118335.0,4114.0,173562.0,-11.98,59.0,M-LC,3.86,99.0,0.03,1.21,8.75,XR,NTT,IT
38,INDIAMART,4810.62,-0.50,0.04,106.65,106.72,131584.0,43.0,123379.0,-50.44,38.0,H-SC,3.11,139.0,0.00,0.86,23.42,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.18,2.43,68.18,72.27,118335.0,4114.0,173562.0,-11.98,59.0,M-LC,3.86,99.0,0.03,1.21,8.75,XR,NTT,IT
38,INDIAMART,4810.62,-0.50,0.04,106.65,106.72,131584.0,43.0,123379.0,-50.44,38.0,H-SC,3.11,139.0,0.00,0.86,23.42,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-0.31,-1.12,37.17,35.63,77024.0,-2353.0,207221.0,-14.97,41.0,H-MC,3.97,119.0,-0.03,1.44,15.36,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.47,-4.65,118.04,107.91,166180.0,-6861.0,140783.0,-22.58,36.0,M-SC,10.57,216.0,-0.04,0.98,0.67,OX40N,ATH,CABLES
37,IEX,219.00,-0.54,-4.31,55.32,48.63,105848.0,-8617.0,191337.0,-36.24,52.0,H-SC,15.62,136.0,-0.08,1.33,6.75,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.13,-35.56,109.81,35.20,91051.0,-45753.0,82917.0,8.74,24.0,X-SC,13.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.02,-9.10,26.76,15.23,48734.0,-18232.0,182115.0,-22.32,28.0,X-MC,6.92,66.0,-0.37,1.27,20.22,X40N,NTT,REALTY
76,TMPV,600.00,-0.49,-15.28,67.83,42.18,158832.0,-42248.0,234162.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.43,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.32,-13.84,72.30,48.45,124904.0,-27748.0,172758.0,-20.09,32.0,X-MC,7.32,64.0,-0.22,1.20,0.36,X40N,ATH,FINANCE
55,PGHH,17907.65,0.07,-4.50,40.07,33.76,76844.0,-9045.0,191775.0,-32.64,33.0,X-MC,3.72,57.0,-0.12,1.34,0.46,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.42,-1.29,11.88,10.44,23470.0,-2582.0,197556.0,-41.15,45.0,X-LC,1.01,5.0,-0.11,1.38,4.20,X40,NTT,FMCG
76,TMPV,600.0,-0.49,-15.28,67.83,42.18,158832.0,-42248.0,234162.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.43,XY24,NTT,AUTO
3,ACC,3906.0,-0.93,-21.54,109.21,64.15,203895.0,-51251.0,186700.0,-54.13,52.0,X-SC,1.50,82.0,-0.25,1.30,4.73,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,0.55,3.67,26.33,30.97,48244.0,6490.0,183228.0,-20.69,59.0,X-MC,1.83,75.0,0.13,1.28,16.69,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-0.14,-10.96,40.96,25.51,67873.0,-20390.0,165705.0,1.77,66.0,H-LC,1.86,49.0,-0.30,1.16,22.11,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.13,-35.56,109.81,35.20,91051.0,-45753.0,82917.0,8.74,24.0,X-SC,13.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.32,-13.84,72.30,48.45,124904.0,-27748.0,172758.0,-20.09,32.0,X-MC,7.32,64.0,-0.22,1.20,0.36,X40N,ATH,FINANCE
55,PGHH,17907.65,0.07,-4.50,40.07,33.76,76844.0,-9045.0,191775.0,-32.64,33.0,X-MC,3.72,57.0,-0.12,1.34,0.46,X40,ATH,FMCG
54,PAGEIND,51605.07,1.21,-4.73,32.32,26.07,50419.0,-7740.0,156000.0,-29.81,39.0,X-MC,9.18,55.0,-0.15,1.09,1.21,X40,ATH,APPARELS
76,TMPV,600.00,-0.49,-15.28,67.83,42.18,158832.0,-42248.0,234162.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.43,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.88,-28.24,99.05,42.84,46206.0,-18357.0,46649.0,-53.17,40.0,X-SC,37.58,83.0,-0.40,0.33,1.51,XY24,NTT,MISC
60,RELAXO,1176.00,-0.65,-46.94,187.04,52.31,144066.0,-68136.0,77024.0,-43.47,38.0,X-SC,4.44,91.0,-0.47,0.54,3.10,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.13,-35.56,109.81,35.20,91051.0,-45753.0,82917.0,8.74,24.0,X-SC,13.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.19,-5.93,29.24,21.58,38548.0,-8307.0,131832.0,-52.56,36.0,X-MC,6.58,56.0,-0.22,0.92,16.71,X40N,ATH,IT
52,MEDANTA,1486.00,-1.18,4.24,19.03,24.08,25182.0,5380.0,132330.0,-4.20,48.0,X-SC,5.56,89.0,0.21,0.92,24.69,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.94,-14.86,40.11,19.28,118136.0,-51426.0,294530.0,-25.89,57.0,X-LC,7.24,1.0,-0.44,2.05,8.88,X40,ATH,IT
41,INFY,2275.00,0.57,6.66,45.20,54.87,150845.0,20840.0,333728.0,-17.09,62.0,X-LC,2.62,2.0,0.14,2.33,15.49,X40,BTT,IT
76,TMPV,600.00,-0.49,-15.28,67.83,42.18,158832.0,-42248.0,234162.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.43,XY24,NTT,AUTO
82,VBL,671.64,0.42,-5.58,43.68,35.67,130268.0,-17609.0,298233.0,-16.86,55.0,X-LC,5.76,4.0,-0.14,2.08,7.51,X40N,ATH,FMCG
43,ITC,452.00,0.42,-1.29,11.88,10.44,23470.0,-2582.0,197556.0,-41.15,45.0,X-LC,1.01,5.0,-0.11,1.38,4.20,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.80,-36.69,109.91,32.89,53722.0,-28328.0,48878.0,-703.29,65.0,L-MC,5.65,259.0,-0.53,0.34,34.60,XR,NTT,BANKS
6,ASIANTILES,137.00,0.20,-12.48,107.58,81.67,88327.0,-11706.0,82104.0,7383.33,53.0,L-SC,19.33,271.0,-0.13,0.57,59.00,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.57,-11.75,58.13,39.54,51891.0,-11891.0,89268.0,-31.03,67.0,M-SC,6.60,220.0,-0.23,0.62,25.88,AR,ATH,AUTO
51,MASFIN,398.61,-0.72,-6.52,30.56,22.05,27990.0,-6390.0,91590.0,-19.60,46.0,H-SC,6.95,164.0,-0.23,0.64,33.13,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.92,-19.83,53.47,23.03,76759.0,-35516.0,143555.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.43,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.20,-9.22,10.18,0.02,16132.0,-16093.0,158466.0,141.63,66.0,M-SC,9.14,234.0,-1.00,1.10,37.84,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.17,0.63,20.19,20.95,51150.0,1575.0,253343.0,-0.35,76.0,X-LC,14.54,31.0,0.03,1.77,36.48,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.80,-36.69,109.91,32.89,53722.0,-28328.0,48878.0,-703.29,65.0,L-MC,5.65,259.0,-0.53,0.34,34.60,XR,NTT,BANKS
0,5PAISA,593.00,-1.05,-35.22,74.28,12.90,105139.0,-76952.0,141544.0,109.13,57.0,H-SC,13.42,161.0,-0.73,0.99,17.69,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,-0.84,6.31,13.60,20.77,33853.0,14774.0,248918.0,-2.93,61.0,X-MC,3.76,62.0,0.44,1.74,13.66,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.85
1,25,44.76
2,50,75.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.53
MC    29.21
LC    25.28
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.81
X40      21.25
X40N     11.93
XR        9.53
XY25      9.12
AR        9.10
OX40N     7.64
X200      1.89
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.73
X-LC    20.04
M-SC    11.91
X-SC     8.29
H-MC     4.75
H-LC     3.05
L-SC     1.40
M-MC     1.39
M-LC     1.21
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.83,-4.37,38.13
IT,13.79,-14.36,73.39
FINANCE,9.46,-14.28,62.56
MISC,7.41,-22.14,72.17
ELECTRICAL,6.00,-9.06,49.01
PAINTS,5.97,-7.80,23.82
INSURANCE,4.42,0.37,34.74
PHARMA,3.87,-0.53,32.71
AUTO,3.36,-19.19,70.29


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3047791.0,21
XR,1295014.0,13
AR,1266944.0,10
X40,992242.0,14
X40N,738454.0,9
OX40N,694999.0,10
XY25,333362.0,6
SR,280318.0,2
MH,73698.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3547319.0,25
M-SC,1350560.0,15
X-MC,1222267.0,16
X-LC,890229.0,11
X-SC,731667.0,8
H-MC,395660.0,3
L-SC,259787.0,3
H-LC,121096.0,2
M-LC,118335.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1212417.0      6
           AR         872308.0      5
           XR         791007.0      7
M-SC       XY24       780358.0      6
X-MC       X40        438876.0      7
X-LC       X40        389654.0      5
X-MC       XY24       362216.0      3
H-SC       OX40N      317571.0      4
X-SC       X40N       292672.0      3
M-SC       OX40N      289509.0      5
H-SC       SR         280318.0      2
X-SC       XY24       275283.0      3
X-MC       X40N       246039.0      4
X-LC       XY24       228009.0      2
H-MC       AR         206152.0      2
X-LC       X40N       199743.0      2
H-MC       XY24       189508.0      1
X-MC       XY25       175136.0      2
L-SC       XR         171868.0      2
X-SC       X40        163712.0      2
M-SC       XR         160082.0      2
           AR         120611.0      2
M-LC       XR         118335.0      1
L-SC       OX40N       87919.0      1
H-SC       MH          73698.0      1
X-LC       XY25        72823.0      2
H-LC       AR          67873.0      1
L-MC       XR          53722.0      1
H-LC       X200        53223.0      1
M-MC       XY25        46064.0      1
L-LC       XY25        39339.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
